# Quering SQl database for UAVSAR team
Import all the necessary python libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import datetime

#database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas

# Loading the database for extracting files

In [2]:
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

print('snowexsql database loading successfull!')

snowexsql database loading successfull!


In [3]:
# Query the session using .surveyors() to generate a list
qry = session.query(ImageData.surveyors)

# Locate all that are distinct
airborne_sensors_list = session.query(ImageData.surveyors).distinct().all()

print('list of airborne sensors by "surveyor" name: \n', airborne_sensors_list)

list of airborne sensors by "surveyor" name: 
 [('USGS',), ('UAVSAR team, JPL',), ('ASO Inc.',)]


# Finding Data for UAVSAR Team

In [4]:
# Airborne sensor from list above
sensor = 'UAVSAR team, JPL'

# Form on the Images table that returns Raster collection dates
qry = session.query(ImageData.date)

# Filter for UAVSAR data
qry = qry.filter(ImageData.surveyors == sensor)

# Grab the unique dates
qry = qry.distinct()

# Execute the query 
dates = qry.all() 

# Clean up the dates 
dates = [d[0] for d in dates] 
dlist = [str(d) for d in dates]
dlist = ", ".join(dlist)
print('%s flight dates are: %s' %(sensor, dlist))

# Find all the snow pits done on these days
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(dates))

# Return a geopandas df
df = query_to_geopandas(qry, engine)

# View the returned pandas dataframe!
print(df.head())

# Close your session to avoid hanging transactions
session.close()

UAVSAR team, JPL flight dates are: 2020-01-31, 2020-02-12
                             geom site_id        date
0  POINT (740652.000 4327445.000)     2C2  2020-01-31
1  POINT (744396.000 4323540.000)    8C26  2020-01-31
2  POINT (741960.000 4326644.000)    6C10  2020-01-31
3  POINT (741493.000 4326833.000)     1C8  2020-01-31
4  POINT (745340.000 4322754.000)    8S28  2020-01-31


In [5]:
# Pick a day from the list of dates
dt = dates[0] 

# Find all the snow pits done on these days 
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date == dt)

# Return a geopandas df
df_exact = query_to_geopandas(qry, engine)

print('%s pits overlap with %s on %s' %(len(df_exact), sensor, dt))

# View snows pits that align with first UAVSAR date
df_exact.head()

17 pits overlap with UAVSAR team, JPL on 2020-01-31


,geom,site_id,date
0,POINT (740652.000 4327445.000),2C2,2020-01-31
1,POINT (744396.000 4323540.000),8C26,2020-01-31
2,POINT (741960.000 4326644.000),6C10,2020-01-31
3,POINT (741493.000 4326833.000),1C8,2020-01-31
4,POINT (745340.000 4322754.000),8S28,2020-01-31


# Form a date range to query on either side of our chosen day as the Insar interferometry date that is 2020-02-01.

In [8]:
date_range = [dt + i * datetime.timedelta(days=1) for i in [1]]

# Find all the snow pits done on these days 
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(date_range))

# Return a geopandas df
df_range = query_to_geopandas(qry, engine)

# Clean up dates (for print statement only)
dlist = [str(d) for d in date_range]
dlist = ", ".join(dlist)

print('%s pits overlap with %s on %s' %(len(df_range), sensor, dlist))

# View snow pits that are +/- 1 day of the first UAVSAR flight date
df_range.sample(10)

14 pits overlap with UAVSAR team, JPL on 2020-02-01


,geom,site_id,date
10,POINT (745842.000 4323805.000),9N44,2020-02-01
0,POINT (744561.000 4322721.000),5S21,2020-02-01
11,POINT (746802.000 4324222.000),2N48,2020-02-01
13,POINT (742607.000 4322917.000),1S8,2020-02-01
6,POINT (743652.000 4322680.000),3S14,2020-02-01
1,POINT (746228.000 4322671.000),9S39,2020-02-01
2,POINT (745937.000 4322754.000),2S37,2020-02-01
9,POINT (746303.000 4322571.000),9S40,2020-02-01
12,POINT (743109.000 4322924.000),2S11,2020-02-01
8,POINT (746553.000 4323759.000),6N46,2020-02-01
